In [1]:
import numpy as np
import pandas as pd
import queue
import matplotlib.pyplot as plt

import sys
backtest_dir = 'C://backtest/backtest/'
if backtest_dir not in sys.path:
    sys.path.insert(0, backtest_dir)
    
from Backtest import *
from BayesianOptimization import *
from BOLL_RSIStrategy import BOLL_RSIStrategy
from Backtest.open_json_gz_files import open_json_gz_files
from Backtest.generate_bars import generate_bars

In [2]:
def run_backtest(config, trading_data, ohlc_data, window_BOLL, a, window_RSI, s, b):
    window_BOLL = int(window_BOLL)
    window_RSI = int(window_RSI)
    s = int(s)
    b = int(b)
    config['title'] = "BOLL_RSIStrategy" + "_" + str(window_BOLL) + "_" + str(a) + "_" + str(window_RSI) + "_" + str(s) + "_" + str(b)
    print("---------------------------------")
    print(config['title'])
    print("---------------------------------")


    events_queue = queue.Queue()
    data_handler = OHLCDataHandler(
        config, events_queue,
        trading_data = trading_data, ohlc_data = ohlc_data
    )

    strategy = BOLL_RSIStrategy(config, events_queue, data_handler,
                            window_BOLL = window_BOLL, a = a,
                            window_RSI=window_RSI, s=s, b=b)

    backtest = Backtest(config, events_queue, strategy,
                        data_handler= data_handler)

    results = backtest.start_trading()
    
    return (results['cum_returns'][-1] - 1)

## Setting

In [3]:
config = {
    "csv_dir": "C:/backtest/Binance",
    "out_dir": "C:/backtest/results/BOLL_RSIStrategy",
    "title": "BOLL_RSIStrategy",
    "is_plot": False,
    "save_plot": False,
    "save_tradelog": False,
    "start_date": pd.Timestamp("2017-07-01T00:0:00", freq = "60" + "T"),    # str(freq) + "T"
    "end_date": pd.Timestamp("2018-04-01T00:00:00", freq = "60" + "T"),
    "equity": 1.0,
    "freq": 60,      # min
    "commission_ratio": 0.001,
    "suggested_quantity": None,     # None or a value
    "max_quantity": None,           # None or a value, Maximum purchase quantity
    "min_quantity": None,           # None or a value, Minimum purchase quantity
    "min_handheld_cash": None,      # None or a value, Minimum handheld funds
    "exchange": "Binance",
    "tickers": ['BTCUSDT']
}

In [4]:
# trading_data = {}
# for ticker in config['tickers']:
#     # trading_data[ticker] = open_gz_files(config['csv_dir'], ticker)
#     trading_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '.h5', key=ticker)

ohlc_data = {}
for ticker in config['tickers']:
    # ohlc_data[ticker] = generate_bars(trading_data, ticker, config['freq'])
    ohlc_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker +'_OHLC_60min.h5', key=ticker)

trading_data = None

In [5]:
gp_params = {"alpha": 1e-5}

## Acquisition Function "Upper Confidence Bound"

* Prefer exploitation (kappa=1.0)

* Prefer exploration (kappa=10)


## Acquisition Function "Expected Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)

In [6]:
BO = BayesianOptimization(
    run_backtest,
    {'window_BOLL': (1, 120),
     'a': (0.1, 3),
    'window_RSI': (1, 120),
       's': (55, 85),
      'b': (15, 45),
    },
    is_int = [1, 0, 1, 1, 1], 
    invariant = {
        'config': config,
        'trading_data': trading_data,
        'ohlc_data': ohlc_data
    },
    random_state = 1
)
# BO.explore({
#     'short_window': np.arange(1, 120, 20),
#     'delta_window': np.arange(1, 120, 20)
#     },
#     eager=True)
BO.maximize(init_points=80, n_iter=120, acq="ei", xi=0.01, **gp_params)

Initialization
------------------------------------------------------------------------------------------------
 Step |   Time |      Value |         a |         b |         s |   window_BOLL |   window_RSI | 
---------------------------------
BOLL_RSIStrategy_38_2.3889100525093165_78_79_27
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
------------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -3.6728114604
Sortino Ratio: -1.4850829301
Max Drawdown: 0.1941888471
Max Drawdown Duration: 141800
Total Returns: -0.1922322257
Annualized Returns: -0.4524326431
Compound Annual Growth Rate: -0.3949489322
---------------------------------
Trades: 7
Trade Winning: 14.29%
Average Trade: -2.76%
Average Win: 2.27%
Average Loss: -3.60%
Best Trade: 2.27%
Worst Trade: -8.00%
Worst Trade Date: 0 days 12:00:00
Avg Days in Trade: 0 days 07:51:26
---------------------------------
   10 | 00m06s |   -0.19223 | 0.713722 | 41.000000 | 78.000000 | 17.000000 | 88.000000 | 
---------------------------------
BOLL_RSIStrategy_2_0.870085312179456_17_55_27
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------


---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
   19 | 00m10s |    0.00000 | 1.300762 | 21.000000 | 55.000000 | 102.000000 | 47.000000 | 
---------------------------------
BOLL_RSIStrategy_19_2.1137604574104607_1_83_42
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Rati

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
   28 | 00m07s |    0.00000 | 2.719866 | 39.000000 | 77.000000 | 69.000000 | 3.000000 | 
---------------------------------
BOLL_RSIStrategy_88_0.4986766420240888_85_61_20
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
   38 | 00m06s |    0.00000 | 2.661588 | 41.000000 | 73.000000 | 10.000000 | 29.000000 | 
---------------------------------
BOLL_RSIStrategy_8_1.9086494004612657_95_67_35
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio

...

---------------------------------
BOLL_RSIStrategy_113_0.5588820319238845_5_59_27
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.4562145850
Sortino Ratio: 1.1248860856
Max Drawdown: 0.1487534207
Max Drawdown Duration: 219800
Total Returns: 0.1805976158
Annualized Returns: 0.4250497352
Compound Annual Growth Rate: 0.4780771884
---------------------------------
Trades: 66
Trade Winning: 71.21%
Average Trade: 0.48%
Average Win: 1.57%
Average Loss: -2.23%
Best Trade: 5.95%
Worst Trade: -9.65%
Worst Trade Date: 0 days 07:00:00
Avg Days in Trade: 0 days 06:02:44
---------------------------------
  162 | 00m19s |    0.18060 | 0.558882 | 27.000000 | 59.000000 | 113.000000 | 5.000000 | 
---------------------------------
BOL

---------------------------------
BOLL_RSIStrategy_115_2.4783013280069164_87_60_34
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  171 | 00m25s |    0.00000 | 2.478301 | 34.000000 | 60.000000 | 115.000000 | 87.000000 | 
---------------------------------
BOLL_RSIStrategy_54_0.6163962436605974_62_60_32
--------------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  180 | 00m07s |    0.00000 | 2.456219 | 43.000000 | 56.000000 | 1.000000 | 118.000000 | 
---------------------------------
BOLL_RSIStrategy_104_1.4007384194375632_69_63_26
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Rat

---------------------------------
BOLL_RSIStrategy_67_0.23376356023661848_64_59_38
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  190 | 00m22s |    0.00000 | 0.233764 | 38.000000 | 59.000000 | 67.000000 | 64.000000 | 
---------------------------------
BOLL_RSIStrategy_63_1.8330763930357328_89_61_15
---------------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  199 | 00m39s |    0.00000 | 0.100000 | 15.000000 | 85.000000 | 83.000000 | 120.000000 | 
---------------------------------
BOLL_RSIStrategy_68_2.5803144425497564_66_65_40
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Rat

In [7]:
print(BO.res['max'])

{'max_val': 0.42685582816182976, 'max_params': {'window_BOLL': 30.0, 'a': 0.1, 'window_RSI': 7.0, 's': 76.0, 'b': 45.0}}


In [8]:
BO.maximize(init_points=0, n_iter=120, acq="ei", xi=0.0001, **gp_params)

Bayesian Optimization
------------------------------------------------------------------------------------------------
 Step |   Time |      Value |         a |         b |         s |   window_BOLL |   window_RSI | 
---------------------------------
BOLL_RSIStrategy_12_0.18112833130580613_64_74_31
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
----

C:\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00017873]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 53, 'nit': 3, 'warnflag': 2}
  " state: %s" % convergence_dict)


---------------------------------
BOLL_RSIStrategy_73_0.20703310991026697_105_74_30
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  203 | 00m48s |    0.00000 | 0.207033 | 30.000000 | 74.000000 | 73.000000 | 105.000000 | 
---------------------------------
BOLL_RSIStrategy_73_0.8616952938044824_78_80_30
-------------

---------------------------------
BOLL_RSIStrategy_102_3.0_1_55_31
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 2.4733251784
Sortino Ratio: nan
Max Drawdown: 0.0027890889
Max Drawdown Duration: 200
Total Returns: 0.0391507904
Annualized Returns: 0.0921442567
Compound Annual Growth Rate: 0.0945969739
---------------------------------
Trades: 5
Trade Winning: 100.00%
Average Trade: 0.98%
Average Win: 0.98%
Average Loss: nan%
Best Trade: 2.25%
Worst Trade: 0.41%
Worst Trade Date: 0 days 02:00:00
Avg Days in Trade: 0 days 01:36:00
---------------------------------
  212 | 01m00s |    0.03915 | 3.000000 | 31.000000 | 55.000000 | 102.000000 | 1.000000 | 
---------------------------------
BOLL_RSIStrategy_120_3.0_1_68_38


C:\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([5.66366398e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 51, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)


...

C:\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-1.61765357e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 54, 'nit': 2, 'warnflag': 2}
  " state: %s" % convergence_dict)


---------------------------------
BOLL_RSIStrategy_82_2.109001754238038_78_72_17
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  317 | 00m34s |    0.00000 | 2.109002 | 17.000000 | 72.000000 | 82.000000 | 78.000000 | 


C:\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.00030135]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 63, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


---------------------------------
BOLL_RSIStrategy_91_1.2129869148263082_73_66_22
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-04-01 00:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Sortino Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
Annualized Returns: 0.0000000000
Compound Annual Growth Rate: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: nan%
Average Trade: nan%
Average Win: nan%
Average Loss: nan%
Best Trade: nan%
Worst Trade: nan%
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
  318 | 00m35s |    0.00000 | 1.212987 | 22.000000 | 66.000000 | 91.000000 | 73.000000 | 
---------------------------------
BOLL_RSIStrategy_88_1.1148817203150287_38_78_15
----------------

C:\Anaconda3\lib\site-packages\sklearn\gaussian_process\gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([0.00032379]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 59, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


In [9]:
print(BO.res['max'])

{'max_val': 0.42685582816182976, 'max_params': {'window_BOLL': 30.0, 'a': 0.1, 'window_RSI': 7.0, 's': 76.0, 'b': 45.0}}


In [10]:
Target = pd.DataFrame({'Parameters': BO.X.tolist(), 'Target': BO.Y})
# Target.to_csv(config['out_dir'] + "/target_ei.csv")
Target.sort_values(by = "Target")

,Parameters,Target
21,"[12.0, 0.24486503094365275, 64.0, 69.0, 44.0]",-0.510231
257,"[92.0, 0.1, 1.0, 85.0, 45.0]",-0.420620
14,"[7.0, 0.5255128672268494, 68.0, 79.0, 44.0]",-0.387045
207,"[20.0, 0.37127026687795506, 21.0, 82.0, 44.0]",-0.383778
204,"[60.0, 0.1, 1.0, 55.0, 45.0]",-0.271798
10,"[2.0, 0.870085312179456, 17.0, 55.0, 27.0]",-0.266192
9,"[17.0, 0.7137215364001712, 88.0, 78.0, 41.0]",-0.192232
281,"[6.0, 1.6398260598047296, 42.0, 74.0, 38.0]",-0.141510
122,"[103.0, 0.8879334608983327, 11.0, 72.0, 24.0]",-0.117284
92,"[8.0, 0.32947551951667275, 63.0, 59.0, 38.0]",-0.115948


## Acquisition Function "Probability of Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)